In [1]:
%cd ..

/Users/floydluo/Library/CloudStorage/GoogleDrive-jjluo@terpmail.umd.edu/.shortcut-targets-by-id/1qNzMmGHCg5Xa63Vw3aKbZdMXvkfT2CgC/MedStar/MS_CODE/FieldNN


# Simulate Data


Given a field name, and first 2 dimensional numbes, simulate tensor.

In [2]:
from fieldnn.utils.layerfn import traverse
from fieldnn.utils.simulate import get_next_info, get_simulated_tensor_from_fldname

# Holder

1. Current Layer:
```python
leng_mask = info_idx == 0 # or info_idx != 0
leng = leng_mask.sum(-1)
```

2. Transfer

```python
old_leng = leng
```


3. Next Layer
```python
leng_mask = old_leng != 0
leng = (leng_mask == 0).sum(-1)
```

In [3]:
# leng_mask = info_idx == 0

import torch

def get_Layer2Holder(fullname, holder, Ignore_PSN_Layers = ['B', 'P']):
    # holder = holder
    d = {}
    for layername in list(reversed(fullname.split('-'))):
        if layername in Ignore_PSN_Layers: continue
        leng_mask = holder == 0
        leng = (leng_mask == 0).sum(-1)
        psn_idx = (leng_mask == False).cumsum(-1).masked_fill(leng_mask, 0)
        d[layername] = {'holder': holder, 
                        'leng_mask': leng_mask, 
                        'leng': leng, 
                        'psn_idx': psn_idx}
        # d[layername] = holder, psn_idx
        holder = leng
    Layer2Hoder = d
    return Layer2Hoder

In [4]:

def align_psn_idx(source_layer, current_layer, Layer2Idx, Layer2Holder):
    if source_layer == current_layer:
        psn_idx = Layer2Holder[current_layer]['psn_idx']
        return psn_idx
    else:
        source_psn_idx = Layer2Holder[source_layer]['psn_idx']
        current_leng_mask = Layer2Holder[current_layer]['leng_mask']
        gaps = Layer2Idx[current_layer] - Layer2Idx[source_layer]
        # print(gaps)
        # print(layername)
        # print(prev_info.shape)
        # print(leng_mask.shape)
        # print(leng.shape)
        # print(psn_idx.shape)
        shape0 = list(source_psn_idx.shape) + [1] * gaps
        shape1 = current_leng_mask.shape
        psn_idx = source_psn_idx.view(*shape0).expand(shape1).masked_fill(current_leng_mask, 0)
        # print(cpsn_idx.shape)
        return psn_idx

# Simulate Tensor

In [5]:
# import torch
# # ======= within forward

# ###############
# B_lenP = 3
# B2P_lnEC = [6, 5, 2] # 
# prefix_layers_num = 2
# vocab_size = 5001
# Ignore_PSN_Layers = ['B', 'P']
# ###############


# full_recfldgrn = 'B-P-EC-Diag:Value-DiagVdftGrn' # full_recflgrn.

# info_idx = get_simulated_tensor_from_fldname(full_recfldgrn, 
#                                              B_lenP, B2P_lnEC, 
#                                              prefix_layers_num, vocab_size)
# print(info_idx.shape)
# holder = torch.LongTensor(info_idx)


# ############### gsn_embeddings
# Layer2Idx = {v:idx for idx, v in enumerate(full_recfldgrn.split('-'))}
# name = full_recfldgrn.split('-')[-1]
# Layer2Holder = get_Layer2Holder(full_recfldgrn, holder, Ignore_PSN_Layers)
# psn_layers = list(reversed([i for i in Layer2Idx if i not in Ignore_PSN_Layers]))


# print(Layer2Idx)
# print(name)
# print([i for i in Layer2Holder], '<---- Layer2Holder')
# print(psn_layers, '<---- psn_layers')

# for source_layer in psn_layers:
#     cpsn_idx = align_psn_idx(source_layer, name, Layer2Idx, Layer2Holder)
#     print(source_layer, cpsn_idx.shape)

# Real Tensor

In [6]:
import os
from recfldgrn.datapoint import load_df_data_from_folder
from recfldgrn.utils import convert_relational_list_to_numpy

###################### take this as given
batch_PID_order = ['P0', 'P2', 'P1', 'P3', 'P4', 'P5', 'P6', 'P7']
######################

TaskTensor_folder = 'data/ProcData/TensorFolder/Task2YearXXX'
print(TaskTensor_folder)

l = sorted([i for i in os.listdir(TaskTensor_folder) if 'Grn' in i])
l

data/ProcData/TensorFolder/Task2YearXXX


['A1C@DT-DTDftGrn',
 'A1C@V-A1CNumeDftGrn',
 'Diag@DT-DTDftGrn',
 'Diag@Value-DiagDftGrn',
 'EC@BasicInfo-BasicDftGrn',
 'EC@DT_min-DTDftGrn',
 'P@age-AgeNumeDftGrn',
 'P@basicInfo-basicInfoDftGrn',
 'PNSectSent@Sentence-Tk@TknzLLMGrn']

In [9]:
recfldgrn_list = [
                  'A1C@V-A1CNumeDftGrn',
                  'Diag@DT-DTDftGrn',
                  'Diag@Value-DiagDftGrn',
                  'PNSectSent@Sentence-Tk@TknzLLMGrn'
                ]

In [10]:
batch_rfg = {}

for recfldgrn in recfldgrn_list:
    
    # (1) get tensor_folder
    tensor_folder = os.path.join(TaskTensor_folder, recfldgrn)

    # (2) get df_Pat and full_recfldgrn
    df_Pat = load_df_data_from_folder(tensor_folder).set_index('PID')
    full_recfldgrn = df_Pat.columns[0]
    suffix = full_recfldgrn.split('_')[-1]
    assert recfldgrn in full_recfldgrn

    # (3) load batch: TODO: convert this to DataSet and DataLoader
    df_batch = df_Pat.loc[batch_PID_order]

    # (4) tensor batch as tensor_idx
    new_full_recfldgrn = 'B-' + full_recfldgrn
    values_list = df_batch[full_recfldgrn].to_list()
    suffix = full_recfldgrn.split('_')[-1]
    # print(suffix)
    # print(new_full_recfldgrn)
    D = convert_relational_list_to_numpy(values_list, new_full_recfldgrn, suffix)
    tensor_idx = D[new_full_recfldgrn]
    
    batch_rfg[new_full_recfldgrn] = tensor_idx
    
for k, v in batch_rfg.items(): print(k, v.shape)

B-P-EC-A1C@V-A1CNumeDftGrn_wgt (8, 87, 1, 37)
B-P-EC-Diag@DT-DTDftGrn_idx (8, 87, 22, 7)
B-P-EC-Diag@Value-DiagDftGrn_idx (8, 87, 22, 3)
B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx (8, 87, 1, 15, 121, 11)


## Analyze One RecFldGrn

In [13]:
full_recfldgrn = 'B-P-EC-Diag@Value-DiagDftGrn_idx'
info_idx = batch_rfg[full_recfldgrn]
print(info_idx.shape)
holder = torch.LongTensor(info_idx)

(8, 87, 22, 3)


In [15]:
# full_recfldgrn

def parse_full_recfldgrn(full_recfldgrn):
    # suffix = full_recfldgrn.split('_')[-1]
    recfld = [i for i in full_recfldgrn.split('-') if '@' in i][0]
    rec, fld = recfld.split('@')
    grn_suffix = [i for i in full_recfldgrn.split('-') if 'Grn' in i][0]
    grn, suffix = grn_suffix.split('_')
    prefix_ids = [i for i in full_recfldgrn.split('-') if 'Grn' not in i and '@' not in i]
    return prefix_ids, rec, fld, grn, suffix


prefix_ids, rec, fld, grn, suffix = parse_full_recfldgrn(full_recfldgrn)
recfldgrn = rec + '@' + fld + '-' + grn
print(prefix_ids)
print(rec)
print(fld)
print(grn)
print(suffix)
print(recfldgrn)

['B', 'P', 'EC']
Diag
Value
DiagDftGrn
idx
Diag@Value-DiagDftGrn


# Expander Core

Expander try to convert GRN_idx (or GRN_wgt) to embedding vectors.

It is from idx to vector.

It is a type of order-increase. 


Here the core include: CateEmbedding, NumeEmbedding, and LLMEmbed. 



In [17]:
full_recfldgrn = 'B-P-EC-Diag@Value-DiagDftGrn_idx'
info_idx = batch_rfg[full_recfldgrn]
print(info_idx.shape)
holder = torch.LongTensor(info_idx)
prefix_ids, rec, fld, grn, suffix = parse_full_recfldgrn(full_recfldgrn)
recfldgrn = rec + '@' + fld + '-' + grn

(8, 87, 22, 3)


In [18]:
import os
import pandas as pd

fldgrn_folder = 'data/ProcData/FldGrnInfo'
fullfldgrn_file = os.path.join(fldgrn_folder, rec + '.p')
df_FieldGrainInfo = pd.read_pickle(fullfldgrn_file)
v2idx = df_FieldGrainInfo[df_FieldGrainInfo['recfield2grain'] == recfldgrn].iloc[0]['Vocab']['v2idx']
# TODO: also adding padding to v2idx
print(len(v2idx))

801


In [19]:
# v2idx

In [11]:
############### gsn_embeddings
Ignore_PSN_Layers = ['B', 'P']
prefix_layers_num = len(Ignore_PSN_Layers)
##################

Layer2Idx = {v:idx for idx, v in enumerate(full_recfldgrn.split('-'))}
name = full_recfldgrn.split('-')[-1]
Layer2Holder = get_Layer2Holder(full_recfldgrn, holder, Ignore_PSN_Layers)
psn_layers = list(reversed([i for i in Layer2Idx if i not in Ignore_PSN_Layers]))

print(Layer2Idx)
print(name)
print([i for i in Layer2Holder], '<---- Layer2Holder')
print(psn_layers, '<---- psn_layers')

for source_layer in psn_layers:
    cpsn_idx = align_psn_idx(source_layer, name, Layer2Idx, Layer2Holder)
    print(source_layer, cpsn_idx.shape)

{'B': 0, 'P': 1, 'EC': 2, 'Diag@Value': 3, 'DiagDftGrn_idx': 4}
DiagDftGrn_idx
['DiagDftGrn_idx', 'Diag@Value', 'EC'] <---- Layer2Holder
['DiagDftGrn_idx', 'Diag@Value', 'EC'] <---- psn_layers
DiagDftGrn_idx torch.Size([8, 87, 22, 3])
Diag@Value torch.Size([8, 87, 22, 3])
EC torch.Size([8, 87, 22, 3])


## CateEmbedding (Expander)

### Module

In [20]:
# fieldlm.nn.embedding
import os
import torch
import numpy as np

class CateEmbeddingLayer(torch.nn.Module):

    def __init__(self, 
                 input_size, 
                 embedding_size, 
                 init = 'random', 
                 freeze = False):
        
        super(EmbeddingLayer, self).__init__()
        
        # create embedding
        if init == 'random':
            # c. initial from random initialization
            self.embedding = torch.nn.Embedding(input_size, embedding_size, padding_idx = 0)
        
        elif type(init) == np.ndarray:
            # a. load from pretrained array. Here init is an array.
            weight = torch.FloatTensor(init)
            assert weight.shape == (input_size, embedding_size)
            self.embedding = torch.nn.Embedding.from_pretrained(weight, freeze = freeze)
            
        elif os.path.isfile(init):
            # b. load from the pretrained array file.
            weight = torch.FloatTensor(np.load(init))
            assert tuple(weight.shape) == (input_size, embedding_size)
            self.embedding = torch.nn.Embedding.from_pretrained(weight, freeze = freeze)
        
        else:
            raise ValueError(f'In correct init method "{init}"')
        
    def forward(self, info):
        # info is the grain
        info = self.embedding(info)
        return info

### Config

In [22]:
embed_size = 512

In [21]:
# [i for i in Layer2Holder]

In [16]:
# Layer2Holder['EC']

In [17]:

vocab_size = 5000 # Need to be updated.

embed_para =  {'embedding_size': embed_size,
               'init': 'random', 
               'input_size': vocab_size + 1 } # 1:the size of special tokens
embed_para

{'embedding_size': 512, 'init': 'random', 'input_size': 5001}

In [9]:
def generate_psn_embed_para(layername, embed_size):
    
    if 'Grn' not in layername: 
        vocab_size = 100
    else:
        vocab_size = 512
        
    embed_para = {'embedding_size': embed_size,
                  'init': 'random', 
                  'input_size': vocab_size + 1 }
    return embed_para

In [10]:
d = {}
d[fullname] = embed_para



psn_layers = [i for i in Layer2Holder]

for layername in psn_layers:
    if layername == 'P': break
    embed_para = generate_psn_embed_para(layername, embed_size)
    d[f'{layername}_psn'] = generate_psn_embed_para(layername, embed_size)
    
d

{'B-PatRec:EC-ECRec:Diag-DiagRec:DiagV-DiagVdftGrn': {'embedding_size': 512,
  'init': 'random',
  'input_size': 5001},
 'DiagVdftGrn_psn': {'embedding_size': 512,
  'init': 'random',
  'input_size': 513},
 'DiagRec:DiagV_psn': {'embedding_size': 512,
  'init': 'random',
  'input_size': 101},
 'ECRec:Diag_psn': {'embedding_size': 512,
  'init': 'random',
  'input_size': 101},
 'PatRec:EC_psn': {'embedding_size': 512, 'init': 'random', 'input_size': 101}}

In [11]:
for nn_name, layer_para in d.items():
    print(nn_name)
    print(layer_para)

B-PatRec:EC-ECRec:Diag-DiagRec:DiagV-DiagVdftGrn
{'embedding_size': 512, 'init': 'random', 'input_size': 5001}
DiagVdftGrn_psn
{'embedding_size': 512, 'init': 'random', 'input_size': 513}
DiagRec:DiagV_psn
{'embedding_size': 512, 'init': 'random', 'input_size': 101}
ECRec:Diag_psn
{'embedding_size': 512, 'init': 'random', 'input_size': 101}
PatRec:EC_psn
{'embedding_size': 512, 'init': 'random', 'input_size': 101}


### Usage 

In [12]:
NN_Dict = {}

for nn_name, layer_para in d.items():
    layer = EmbeddingLayer(**layer_para)
    NN_Dict[nn_name] = layer
    
NN_Dict

{'B-PatRec:EC-ECRec:Diag-DiagRec:DiagV-DiagVdftGrn': EmbeddingLayer(
   (embedding): Embedding(5001, 512, padding_idx=0)
 ),
 'DiagVdftGrn_psn': EmbeddingLayer(
   (embedding): Embedding(513, 512, padding_idx=0)
 ),
 'DiagRec:DiagV_psn': EmbeddingLayer(
   (embedding): Embedding(101, 512, padding_idx=0)
 ),
 'ECRec:Diag_psn': EmbeddingLayer(
   (embedding): Embedding(101, 512, padding_idx=0)
 ),
 'PatRec:EC_psn': EmbeddingLayer(
   (embedding): Embedding(101, 512, padding_idx=0)
 )}

In [13]:
import torch

########################
B_lenP = 3
B2P_lnEC = [6, 4, 3] # 
prefix_layers_num = 2
vocab_size = 100
########################

fullname = 'B-PatRec:EC-ECRec:Diag-DiagRec:DiagV-DiagVdftGrn'
layer2layeridx = {v:idx for idx, v in enumerate(fullname.split('2'))}
name = fullname.split('-')[-1]

data = get_simulated_tensor_from_fldname(fullname, B_lenP, B2P_lnEC, prefix_layers_num, vocab_size)
print(data.shape)
# fld_tensor_idx


info_idx = torch.LongTensor(data)
print(info_idx.shape)
holder = info_idx
# grn_leng_mask = info_idx == 0
# print(grn_leng_mask.shape)

Layer2Holder = get_Layer2Holder(fullname, holder)
for i in Layer2Holder: print(i)

1
1 --> (3,)
2 --> (3, 6)
2
2 --> (3, 6)
3 --> (3, 6, 6)
3
3 --> (3, 6, 6)
4 --> (3, 6, 6, 8)
(3, 6, 6, 8)
torch.Size([3, 6, 6, 8])
DiagVdftGrn
DiagRec:DiagV
ECRec:Diag
PatRec:EC


In [14]:
# embed
Embed = NN_Dict[fullname]
Embed

EmbeddingLayer(
  (embedding): Embedding(5001, 512, padding_idx=0)
)

In [15]:
info = Embed(info_idx)
# tensor_name = tensor_name.replace('_idx', '2Feat_flt')
print(info.shape)
# print(tensor_name)

torch.Size([3, 6, 6, 8, 512])


In [16]:
# tensor_name.replace('_idx', '2Feat_flt')

## NumeEmbedding (Expander)

## LLMEmbedding (Expander)

TODO: adding huggingface

# Expander

In [ ]:
import torch
import torch.nn.functional as F

# from fieldnn.nn.embedding import EmbeddingLayer
# from fieldnn.nn.lmembed import LMEmbedLayer
# from fieldnn.utils.layer import get_Layer2Holder, align_psn_idx

# from ..nn.embedding import EmbeddingLayer
# from ..nn.lmembed import LMEmbedLayer
# from ..utils.layerfn import get_Layer2Holder, align_psn_idx


class Expander_Layer(torch.nn.Module):
    
    '''Only for Increasing Embedding Dimensions'''
    def __init__(self, input_fullname, output_fullname, expander_layer_para):
        super(Expander_Layer, self).__init__()
        
        # tensor.shape[-1]
        self.input_size = expander_layer_para['input_size']
        self.output_size = expander_layer_para['output_size']
        self.Ignore_PSN_Layers = expander_layer_para['Ignore_PSN_Layers']
        
        # Part 1: embedding
        self.input_fullname = input_fullname
        self.output_fullname = output_fullname
        
        assert 'Grn' == input_fullname[-3:]
        assert input_fullname.replace('Grn', '') == output_fullname
        
        nn_name, para = expander_layer_para[input_fullname]
        
        if nn_name.lower() == 'embedding':
            self.Embed = EmbeddingLayer(**para)
            self.embed_size = para['embedding_size']
        elif nn_name.lower() == 'lmembed':
            # TODO
            self.Embed = LMEmbedLayer(**para)
            self.embed_size = para['embedding_size']
        else:
            raise ValueError(f'NN "{nn_name}" is not available')
            
        assert self.embed_size == self.output_size
        
        # Part 2: PSN embedding
        # psn_layers = expander_layer_para['psn_layers']
        # self.PSN_Embed_Dict = torch.nn.ModuleDict()
        # for layername in psn_layers:
        #     para = generate_psn_embed_para(layername, self.embed_size)
        #     self.PSN_Embed_Dict[layername] = EmbeddingLayer(**para)
        
        # Part 3: PostProcess
        self.postprocess = torch.nn.ModuleDict()
        for method, config in expander_layer_para['postprocess'].items():
            if method == 'dropout':
                self.postprocess[method] = torch.nn.Dropout(**config)
            elif method == 'activator':
                activator = config
                if activator.lower() == 'relu': 
                    self.postprocess[method] = torch.nn.ReLU()
                elif activator.lower() == 'tanh': 
                    self.postprocess[method] = torch.nn.Tanh()
                elif activator.lower() == 'gelu':
                    self.postprocess[method] = torch.nn.GELU()
            elif method == 'layernorm':
                self.postprocess[method] = torch.nn.LayerNorm(self.embed_size, **config)
                
    # def get_psn_embed(self, fullname, holder):
    #     name = fullname.split('-')[-1]
    #     Layer2Idx = {v:idx for idx, v in enumerate(fullname.split('-'))}
    #     Layer2Holder = get_Layer2Holder(fullname, holder, self.Ignore_PSN_Layers)
        
    #     psn_embed = 0
    #     for source_layer, Embed in self.PSN_Embed_Dict.items():
    #         cpsn_idx = align_psn_idx(source_layer, name, Layer2Idx, Layer2Holder)
    #         psn_embed = psn_embed + Embed(cpsn_idx)
        
    #     return psn_embed

    def forward(self, fullname, holder, info = 'Empty'):
        '''
            fullname: full name of field, GRN is here
            holder: info_idx
            info: info_wgt
        '''
        assert self.input_fullname == fullname
        leng_mask = holder == 0
        embed = self.Embed(holder)
        
        if type(info) != str:
            embed = embed * info.unsqueeze(-1)
        
        # Comments: move these to Learner.
        # if len(self.PSN_Embed_Dict):
        #     psn_embed = self.get_psn_embed(fullname, holder)
        #     embed = embed + psn_embed
        
        for nn, layer in self.postprocess.items():
            embed = layer(embed)
        
        return self.output_fullname, holder, embed

# Transformer (Learner)

## Module

In [17]:
import torch
import numpy as np
import torch.nn.functional as F


class TFMLayer(torch.nn.Module):
    def __init__(self, 
                 input_size = 512, 
                 output_size = 512, # d_model
                 nhead = 8,
                 num_encoder_layers = 6, # only have encoder part
                 num_decoder_layers = 0, # in default, we don't need decoder part. 
                 dim_feedforward = 2048, 
                 tfm_dropout = 0.1,
                 tfm_activation = 'relu'):
        
        '''https://github.com/pytorch/pytorch/blob/master/torch/nn/modules/transformer.py'''

        super(TFMLayer,self).__init__()
        self.num_encoder_layers = num_encoder_layers
        self.num_decoder_layers = num_decoder_layers
        self.input_size = input_size
        self.tfm_input_size = input_size
        self.n_directions = 1
        self.output_size = output_size
        assert output_size % self.n_directions == 0 
        self.hidden_size = int(output_size / self.n_directions)
        assert self.hidden_size == self.tfm_input_size
            
        self.transformer  = torch.nn.Transformer(d_model = self.hidden_size, 
                                                 nhead = nhead,
                                                 num_encoder_layers = self.num_encoder_layers,
                                                 num_decoder_layers = self.num_decoder_layers,
                                                 dim_feedforward = dim_feedforward, 
                                                 dropout = tfm_dropout,
                                                 activation = tfm_activation,
                                                 batch_first = True,
                                                 # src_mask_flag = False, # see all tokens in a sentence 
                                                 # # This IS THE NEW PART. NOT PyTorch.nn.
                                                 ) 


    def forward(self, info, leng_mask):
        info = self.transformer(info, info, src_key_padding_mask = leng_mask,  tgt_key_padding_mask  = leng_mask)
        # for layer in self.postprocess:
        #     info = layer(info)
        return info

## Config

In [18]:
tfm_para =  {'input_size': 512,
             'output_size': 512,
             'nhead': 8,
             'num_encoder_layers': 6,
             'num_decoder_layers': 0,
             'dim_feedforward': 2048,
             'tfm_dropout': 0.1,
             'tfm_activation': 'relu'}
tfm_para

{'input_size': 512,
 'output_size': 512,
 'nhead': 8,
 'num_encoder_layers': 6,
 'num_decoder_layers': 0,
 'dim_feedforward': 2048,
 'tfm_dropout': 0.1,
 'tfm_activation': 'relu'}

## Usage

In [19]:
tfm_layer = TFMLayer(**tfm_para)
tfm_layer

TFMLayer(
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-5): 6 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
          )
          (linear1): Linear(in_features=512, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=2048, out_features=512, bias=True)
          (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
      )
      (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    )
    (decoder): TransformerDecoder(
      (layers): ModuleList()
      (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    )
  )
)

In [20]:
print(fullname)
print(info_idx.shape)
info = Embed(info_idx)
print(info.shape)

B-PatRec:EC-ECRec:Diag-DiagRec:DiagV-DiagVdftGrn
torch.Size([3, 6, 6, 8])
torch.Size([3, 6, 6, 8, 512])


In [21]:
print(info_idx.shape)
leng_mask = info_idx == 0
print(info.shape)
print(leng_mask.shape)

torch.Size([3, 6, 6, 8])
torch.Size([3, 6, 6, 8, 512])
torch.Size([3, 6, 6, 8])


### Reshape

In [24]:
# 3*6*6

In [25]:
nbs = np.array(info.shape[:-2]).prod()
ngrn, dim = info.shape[-2:]
print(nbs, ngrn, dim)


tmp_info = info.contiguous().view(nbs, ngrn, dim)
print(tmp_info.shape)

tmp_leng_mask = leng_mask.contiguous().view(nbs, ngrn)
print(tmp_leng_mask.shape)

tmp_leng = (tmp_leng_mask == 0).sum(-1)
print(tmp_leng.shape)

108 8 512
torch.Size([108, 8, 512])
torch.Size([108, 8])
torch.Size([108])


In [26]:
# order sequences and restore sequences according to their lenghts
# TODO: test the speed of orderSeq and restoreSeq

def orderSeq(seq_unordered, leng_unordered):
    # leng_unordered is a tensor
    # seq_unordered is a numpy
    leng_ordered, seq_index = leng_unordered.sort(descending=True) 
    _, reverse_index = seq_index.sort()
    leng_ordered = leng_ordered[leng_ordered>0]
    seq_index    = seq_index[:len(leng_ordered)]
    seq_ordered  = seq_unordered[seq_index.cpu()]
    return seq_ordered, leng_ordered, reverse_index

In [27]:
tmp_info = info.contiguous().view(nbs, ngrn, dim)
print(tmp_info.shape)

ord_info,      ord_leng, r_idx = orderSeq(tmp_info, tmp_leng)
ord_leng_mask, ord_leng, r_idx = orderSeq(tmp_leng_mask, tmp_leng)
print(ord_info.shape)
print(ord_leng.shape)
print(r_idx.shape)
print(ord_leng)

torch.Size([108, 8, 512])
torch.Size([48, 8, 512])
torch.Size([48])
torch.Size([108])
tensor([8, 8, 8, 8, 8, 7, 7, 7, 7, 7, 7, 7, 7, 6, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1])


In [ ]:
# [image1,    image2,    image3]
# [outcome1, outcome2, outcome3]
# 48, 8, 512
# 20, 15, 13
# [s1, s2.....]
# [r1, r2,....]

In [28]:
print(tmp_leng_mask.shape)
ord_leng_mask, ord_leng, r_idx = orderSeq(tmp_leng_mask, tmp_leng)
print(ord_leng_mask.shape)
print(ord_leng.shape)
print(r_idx.shape)
print(ord_leng)

torch.Size([108, 8])
torch.Size([48, 8])
torch.Size([48])
torch.Size([108])
tensor([8, 8, 8, 8, 8, 7, 7, 7, 7, 7, 7, 7, 7, 6, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1])


### Calculate

In [29]:
ord_info_output = tfm_layer(ord_info, ord_leng_mask)
print(ord_info_output.shape)

torch.Size([48, 8, 512])


### Restore

In [30]:
def restoreSeq(seq_ordered, reverse_index):
    # shape = list(seq_ordered.shape)
    data_type = seq_ordered.type()
    shape = list(seq_ordered.shape)
    shape[0] = len(reverse_index) - shape[0]
    t = torch.cat([seq_ordered, torch.zeros(shape).type(data_type)])
    seq_restored = t[reverse_index]
    return seq_restored

In [31]:
info_new = restoreSeq(ord_info_output, r_idx)
print(info_new.shape)

torch.Size([108, 8, 512])


In [32]:
output_size = dim
info_output = info_new.view(*list(leng_mask.shape) + [output_size])
print(info_output.shape)

torch.Size([3, 6, 6, 8, 512])


In [33]:
info_output[:,:,:,:,0]

tensor([[[[ 1.2140,  0.2946,  0.8117,  0.8469,  0.0000,  0.0000,  0.0000,
            0.0000],
          [-0.5250,  0.8469,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
            0.0000],
          [ 0.1241, -1.9581,  0.7282, -1.8982,  1.9860, -1.9581,  1.6688,
            0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
            0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
            0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
            0.0000]],

         [[ 0.6670,  1.9860,  0.0610,  0.0000,  0.0000,  0.0000,  0.0000,
            0.0000],
          [ 1.2140,  0.2701,  1.2046,  0.0610, -0.9479, -0.4609,  0.7881,
            1.0486],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
            0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
            0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  

In [34]:
leng_mask

tensor([[[[False, False, False, False,  True,  True,  True,  True],
          [False, False,  True,  True,  True,  True,  True,  True],
          [False, False, False, False, False, False, False,  True],
          [ True,  True,  True,  True,  True,  True,  True,  True],
          [ True,  True,  True,  True,  True,  True,  True,  True],
          [ True,  True,  True,  True,  True,  True,  True,  True]],

         [[False, False, False,  True,  True,  True,  True,  True],
          [False, False, False, False, False, False, False, False],
          [ True,  True,  True,  True,  True,  True,  True,  True],
          [ True,  True,  True,  True,  True,  True,  True,  True],
          [ True,  True,  True,  True,  True,  True,  True,  True],
          [ True,  True,  True,  True,  True,  True,  True,  True]],

         [[False, False, False, False, False, False, False,  True],
          [ True,  True,  True,  True,  True,  True,  True,  True],
          [ True,  True,  True,  True,  True